In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
!pip install pycaret
from pycaret.regression import *


warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
data = pd.read_csv('/kaggle/input/latest-covid19-india-statewise-data/Latest Covid-19 India Status.csv')
print('Shape of the given Data-', data.shape)
print('Columns in given data are-',data.columns.values)

In [ ]:
#Click show ouput to view the data 

<h1>Data Description</h1>

This dataset contains latest Covid-19 India state-wise data as on September 07, 2021. This dataset can be used to analyze covid condition in India.
This dataset is great for Exploratory Data Analysis

* Attribute Information
* State/UTs - Names of Indian States and Union Territories.
* Total Cases - Total number of confirmed cases
* Active - Total number of active cases
* Discharged - Total number of discharged cases
* Deaths - Total number of deaths
* Active Ratio (%) - Ratio of number of active cases to total cases
* Discharge Ratio (%) - Ratio of number of discharged cases to total cases
* Death Ratio (%) - Ratio of number of deaths to total cases

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.style.background_gradient(cmap="Blues")

In [ ]:
data = data.rename(columns={'State/UTs': 'State'})
data.at[7,'State']='DNHDD'

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x= data['State'],y= data['Total Cases'])
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(17,6))
plots = sns.barplot(x= data['State'],y= data['Active Ratio (%)'])
plt.xticks(rotation=90)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=8, xytext=(0, 8),
                   textcoords='offset points')
 


In [ ]:
plt.figure(figsize=(17,6))
plots = sns.barplot(x= data['State'],y= data['Discharge Ratio (%)'])
plt.xticks(rotation=90)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=8, xytext=(0, 8),
                   textcoords='offset points')
 

In [ ]:
plt.figure(figsize=(17,6))
plots = sns.barplot(x= data['State'],y= data['Death Ratio (%)'])
plt.xticks(rotation=90)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=8, xytext=(0, 8),
                   textcoords='offset points')
 

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(data.corr(),annot=True, square= True, cmap='Blues')

In [ ]:
X = data.drop(['State','Death Ratio (%)'],axis=1)
y = data['Death Ratio (%)']

In [ ]:
data_1 = data.copy()
data_1.drop(['Death Ratio (%)'],axis=1,inplace=True)
data_1.drop(['State'],axis=1,inplace=True)
col_drop = data_1.iloc[:1]
data_1 = data_1.iloc[1:]
Deaths = col_drop.Deaths
col_drop.drop('Deaths',axis=1,inplace=True)

In [ ]:
env = setup(data= data_1,target='Deaths',train_size=0.99,normalize=True,normalize_method='robust',session_id = 69,silent=True)

In [ ]:
model = compare_models(sort='R2')

In [ ]:
tuned_model = tune_model(model,fold=10,optimize='RMSE',early_stopping=True,early_stopping_max_iters=10,choose_better = True)
final_model = finalize_model(tuned_model)
print(final_model)

In [ ]:
from sklearn.linear_model import HuberRegressor
y_train = data_1.Deaths
data_1.drop(['Deaths'],axis=1,inplace=True)
hub = HuberRegressor(alpha=0.001, epsilon=1.7, fit_intercept=True, max_iter=100,
               tol=1e-05, warm_start=False)
hub.fit(data_1,y_train)
pred = hub.predict(col_drop)

In [ ]:
print(f'Predicted Deaths: {int(pred[0])}')
print(f'Actual Casuality caused: {Deaths.values[0]}')